In [7]:
!aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 763104351884.dkr.ecr.us-east-1.amazonaws.com   

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [8]:
!docker build -t 715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310 -f Dockerfile.train .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.2s (3/3)                                          docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 248B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
[+] Building 0.2s (9/9) FINISHED                                 docker:default
 => [internal] load build definition from Dockerfile.train                 0.0s
 => => transferring dockerfile: 248B                                       0.0s
 => [internal] load metadata for 763104351884.dkr.ecr.us-east-1.amazonaws  0.1s
 => [auth] sharing credentials for 763104351884.dkr.ecr.us-east-1.amazona  0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

In [9]:
!docker images

REPOSITORY                                                                  TAG                      IMAGE ID       CREATED         SIZE
715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training            2.12.0-cpu-py310         4df6232325a4   4 minutes ago   6.13GB
715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training            2.12.0-cpu-py310-pippo   4df6232325a4   4 minutes ago   6.13GB
520713654638.dkr.ecr.us-east-1.amazonaws.com/sagemaker-tensorflow-serving   1.12.0-cpu               27aee10a9f6a   5 years ago     377MB


In [10]:
%%sh
# Specify an image name
image_name=tensorflow-training
echo "image_name: ${image_name} ######################"

account=$(aws sts get-caller-identity --query Account --output text)
echo "account: ${account} ######################"

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
echo "region: ${region} ######################"

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310"
echo "fullname: ${fullname} ######################"
aws ecr create-repository --repository-name "${image_name}"

image_name: tensorflow-training ######################
account: 638118468186 ######################
region: us-east-1 ######################
fullname: 638118468186.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310 ######################



An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'tensorflow-training' already exists in the registry with id '638118468186'


CalledProcessError: Command 'b'# Specify an image name\nimage_name=tensorflow-training\necho "image_name: ${image_name} ######################"\n\naccount=$(aws sts get-caller-identity --query Account --output text)\necho "account: ${account} ######################"\n\n# Get the region defined in the current configuration (default to us-west-2 if none defined)\nregion=$(aws configure get region)\necho "region: ${region} ######################"\n\nfullname="${account}.dkr.ecr.${region}.amazonaws.com/${image_name}:2.12.0-cpu-py310"\necho "fullname: ${fullname} ######################"\naws ecr create-repository --repository-name "${image_name}"\n'' returned non-zero exit status 255.

In [11]:
!docker push 715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-pippo

The push refers to repository [715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training]

cbb5be75: Preparing 
0077decb: Preparing 
22878ecf: Preparing 
db4f6696: Preparing 
f529ae49: Preparing 
75f13e59: Preparing 
0c6c24f9: Preparing 
a1e64371: Preparing 
29631b13: Preparing 
3f03efd0: Preparing 
9b75d5ee: Preparing 
370de786: Preparing 
a16f1a7a: Preparing 
fc8740d1: Preparing 
292691fc: Preparing 
6672bf9d: Preparing 
607be3da: Preparing 
afc9b360: Preparing 
f934e4df: Preparing 
f934e4df: Waiting g 
a0538fa4: Preparing 
6d109c54: Preparing 
f0a068df: Preparing 
443b7673: Preparing 
37a36861: Preparing 
ecb4bd81: Preparing 
ecb4bd81: Waiting g 
07d5d0ab: Preparing 
e6c8e003: Preparing 
ab822c1c: Preparing 
3609315f: Preparing 
no basic auth credentials


In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310-pippo',
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters,
    output_path='s3://mybucketuf14/data/output/'
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('s3://mybucketuf14/data/input/house_price_regression_dataset.csv')
print('##### ESTIMATOR FIT COMPLETED')